In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-12-05


Found rundate 2023-12-04 10:00:03 < 2023-12-05 at commit 7a2194f8; returning commit 027d27e


'2023-12-05 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-12-05 10:00:03, NJSP was reporting 560 YTD deaths
Current YTD Deaths (2024-12-05 10:00:05-05:00): 639
Previous year YTD Deaths (adjusted; 2023-12-05 10:00:03): 560.1202270068006
Projected 2024 total: 690.8715798969188
14.1% change


FAUQStats cache miss: 2023, Tue Dec 05 10:00:03 EST 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,Mulberry St,Mulberry St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,NaN,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,322,0.0,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,South Randolphville Road,South Randolphville Road,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,Outwater Ln,Outwater Ln,1.0,0.0,0.0,1.0,0.0,NaN,0.0,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,Washington St,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12857,15,Ocean,1509,Harvey Cedars Boro,NaN,County 607 MP 13.92,1.0,0.0,0.0,1.0,0.0,607,NaN,2023-12-01 16:57:00-05:00
12856,06,Cumberland,0614,Vineland City,NaN,State Highway 55 MP 36.1,1.0,1.0,0.0,0.0,0.0,55,NaN,2023-12-02 14:16:00-05:00
12858,16,Passaic,1602,Clifton City,NaN,Garden State Parkway MP 155.7,1.0,1.0,0.0,0.0,0.0,444,NaN,2023-12-03 03:47:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,28,2,17,4,50
Bergen,15,5,19,2,39
Burlington,29,4,7,1,41
Camden,17,3,15,0,35
Cape May,8,0,0,0,8
Cumberland,18,6,6,0,25
Essex,16,5,26,0,46
Gloucester,17,8,5,0,27
Hudson,9,2,5,1,16


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

92.7% through the year, 7.3% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,16,18,28
Bergen,driver,21,20,15
Burlington,driver,24,26,29
Camden,driver,18,18,17
Cape May,driver,4,4,8
...,...,...,...,...
Salem,crashes,10,11,12
Somerset,crashes,19,21,19
Sussex,crashes,6,6,12


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,56,8,31,2,18
Bergen,41,2,14,5,25
Burlington,46,2,31,6,7
Camden,38,0,17,4,18
Cape May,9,0,8,0,0
Cumberland,29,0,19,7,8
Essex,51,0,18,8,27
Gloucester,30,0,18,11,6
Hudson,16,2,8,2,5


In [9]:
projected.to_csv(PROJECTED_CSV)